<a href="https://colab.research.google.com/github/waldekmaciejko/DETpy/blob/main/bert_tokenizer_using_sentiment140.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://www.tensorflow.org/datasets/catalog/sentiment140

#Importing dependencies

In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [2]:
!pip install bert-for-tf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30509 sha256=3089c81ab590f3d902bce26c8349f33324747669b14bb32ea2aef66907083cb7
  Stored in directory: /root/.cache/pip/wheels/d8/da/50/126d7b8416d9a0e6bf876935c2219a71e72a6529c25e150c56
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19455 sha256=5fade028ebd0a32d0c93348d1871c636d336ca5ef06231539c8a460ec763dd5a
  Stored in directory: /root/.cache/pip/wheels/97/a8/d0/f7419404174976a2686bb98b5c30df01cc71445415f32db9e6
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7891 sha256=29d2c4a4ca23f3414a9f81439ca16da4a79174e61a23451b34555574b5b70099
  Stored in directory: /root/.cache/pip/wheels/69/c8/b3/92666cff9fb312bc3473eaa6b396695b89a7b3e31e908768

In [3]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
tf.__version__

'2.14.0'

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

# Loading files

In [6]:
drive.mount("/content/drive/")

Mounted at /content/drive/


In [13]:
cols = ['sentiment', 'id', 'date', 'query', 'user', 'text']

In [14]:
# data from help.sentiment140.com/for-students

data = pd.read_csv("/content/drive/MyDrive/Datasets/StanfordSentimentAnalysis/training.1600000.processed.noemoticon.csv",
                   header=None,
                   names=cols,
                   engine="python",
                   encoding='latin'
                   )

In [9]:
data.head(5)

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


# Cleaning data

In [10]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [11]:
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet, 'lxml').get_text()
  tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
  tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
  tweet = re.sub(r" +", ' ', tweet)
  return tweet

In [12]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

<ipython-input-11-21be760969b6>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  tweet = BeautifulSoup(tweet, 'lxml').get_text()


In [15]:
np.unique(data.sentiment.values)
data_labels = data.sentiment.values
data_labels[data_labels==4] = 1

## Tokenizer - create BERT layer to have an acces to metadata from tokenizer

In [16]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
                            trainable=False)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [17]:
def encode_sentence(sent):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [18]:
tokenizer.tokenize("My dog loves strawberries")

['my', 'dog', 'loves', 'straw', '##berries']

In [19]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("My dog loves strawberries"))

[2026, 3899, 7459, 13137, 20968]

In [20]:
data_inputs = [encode_sentence(sent) for sent in data_clean]

# Dataset creation - we will create paddet baches

In [22]:
data_with_len = [[sent, data_labels[i], len(sent)] for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])
sorted_all = [(sent_lab[0], sent_lab[1])
                for sent_lab in data_with_len if sent_lab[2] > 2]

In [40]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [ ]:
type(all_dataset)

tensorflow.python.data.ops.flat_map_op._FlatMapDataset

In [ ]:
BATCH_SIZE = 32
# Combines consecutive elements of this dataset into padded batches.
# This transformation combines multiple consecutive elements of the input
# dataset into a single element.
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((None, ), ()))

In [ ]:
NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [21]:
%cd

/content
